# Xception Architecture

Importing libraries and code for loading resized images 

In [0]:

import numpy as np
from keras.datasets import cifar10
import cv2
import keras as k
from keras import models
from matplotlib import pyplot as p
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential,Model
from keras import optimizers
from keras.layers import Dense, Dropout, Activation, Flatten,BatchNormalization, Input,Lambda
from keras.layers import Conv2D, MaxPooling2D,GlobalAveragePooling2D
from keras.applications.inception_v3 import InceptionV3, preprocess_input

from keras.applications.xception import Xception
from keras.applications import VGG16
from keras.layers import LeakyReLU
import pandas as pd
import os 
from PIL import Image

def load_images(path,num_classes):
    #Load images
    
    print('Loading ' + str(num_classes) + ' classes')
    X_train = []
    y_train = []
    #X_train=np.zeros([num_classes*500,227,227,3],dtype='uint8')
    #y_train=np.zeros([num_classes*500], dtype='uint8')

    trainPath=path+'/train/train/bilinear'

    print('loading training images...');

    i=0
    j=0
    annotations={}
    for sChild in os.listdir(trainPath):
        sChildPath = os.path.join(os.path.join(trainPath,sChild),'images')
        annotations[sChild]=j
        for c in os.listdir(sChildPath):
            spath=os.path.join(sChildPath,c)   
            img = cv2.imread(spath)
            img = cv2.resize(img, (139, 139))
            X_train.append(img)
            y_train.append(j)
            i+=1
        j+=1
        if (j >= num_classes):
            break

    print('finished loading training images')

    val_annotations_map = get_annotations_map()

   


    print('loading test images...')
    X_test = []
    y_test = []
    i = 0
    testPath=path +'/val/bilinear/images'
    print(testPath + "abc")
    for sChild in os.listdir(testPath):
        if val_annotations_map[sChild] in annotations.keys():
            sChildPath = os.path.join(testPath, sChild)
            img = cv2.imread(sChildPath)
            img = cv2.resize(img, (139, 139))
            X_test.append(img)
            y_test.append(annotations[val_annotations_map[sChild]])
            i+=1
        else:
            pass


    print('finished loading test images'+str(i))
    np.save('data/saved_npy/tf/y_train.npy',y_train)
    np.save('data/saved_npy/tf/x_train.npy',X_train)
    np.save('data/saved_npy/tf/x_test.npy',X_test)
    np.save('data/saved_npy/tf/y_test.npy',y_test)
    return X_train,y_train,X_test,y_test




Using TensorFlow backend.


In [0]:
def get_annotations_map():
	valAnnotationsPath = 'C:/Users/bhash/Downloads/val_annotations.txt'
	valAnnotationsFile = open(valAnnotationsPath, 'r')
	valAnnotationsContents = valAnnotationsFile.read()
	valAnnotations = {}

	for line in valAnnotationsContents.splitlines():
		pieces = line.strip().split()
		valAnnotations[pieces[0]] = pieces[1]

	return valAnnotations

Loading Images

In [0]:
path='C:/Users/bhash/data'
x_train,y_train,x_test,y_test=load_images(path,200)

Loading 200 classes
loading training images...
finished loading training images
loading test images...
C:/Users/bhash/data/val/bilinear/imagesabc
finished loading test images10000


In [0]:
from glob import glob
def xception_from_disk(path = 'data/my_xception.h5'):
    if glob(path):
        fc_model = load_model(path)
        print('Xception Model Loaded from Disk!')
    else:
        fc_model = fc_layer(features_train)
        fc_model.save(path)
    return fc_model

In [0]:
from glob import glob
def load_traindata():
    if glob('data/saved_npy'):
        y_train = np.load('data/saved_npy/tf/y_train.npy')
        x_train = np.load('data/saved_npy/tf/x_train.npy')
        x_test = np.load('data/saved_npy/tf/x_test.npy')
        y_test = np.load('data/saved_npy/tf/y_test.npy')
        print('Training data loaded from disk!')
    else: 
        print('No saved data found. Creating..')
        x_train, y_train = create_traindata()
    return x_train, y_train, x_test, y_test

In [0]:

x_train, y_train,x_test,y_test = load_traindata()

Training data loaded from disk!


Preproessing data

In [0]:
y_train = k.utils.to_categorical(y_train, 200)
y_test = k.utils.to_categorical(y_test, 200)

In [0]:
x_train = features_train.astype('float32')
x_test = features_test.astype('float32')

x_train /= 255
x_test /= 255

# Methods to create  model and save the features of our model 

In [0]:
def cnn_load_model(model='xception'):
    im_size=139
    if model == 'xception':
        cnn_model = Xception(include_top=True, input_shape=(im_size, im_size, 3), weights='imagenet')
    else:
        cnn_model = InceptionV3(include_top=True, input_shape=(im_size, im_size, 3), weights='imagenet')
    
    for layer in cnn_model.layers:
        layer.trainable = False
    
    inputs = Input((im_size, im_size, 3))
    x = inputs
   # x = Lambda(preprocess_input, name='preprocessing')(x)
    x = cnn_model(x)
    #x = GlobalAveragePooling2D()(x)
    #x = GlobalAveragePooling2D()(x)
    cnn_model = Model(inputs, x)
    cnn_model.summary()
    return cnn_model

def bottleneck_features(model='xception'):
    print('Loading model:',model)
    if glob('data/saved_npy/xception_features_train.npy'):
        print("op")
        features_train = np.load('data/saved_npy/xception_features_train.npy')
        features_test = np.load('data/saved_npy/xception_features_test.npy')
        print('Features loaded from disk!')
    else:
        print("hi")
        xception_m = cnn_load_model(model)
        features_train = xception_m.predict(x_train,batch_size=64, verbose=1)
        features_test = xception_m.predict(x_test,batch_size=64, verbose=1)
        np.save('data/saved_npy/tf/xception_features_train.npy',features_train)
        np.save('data/saved_npy/tf/xceptio_features_test.npy',features_test)
    return features_train, features_test

In [0]:
features_train, features_test = np.asarray(bottleneck_features(model='xception'))

Loading model: xception
hi
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 139, 139, 3)       0         
_________________________________________________________________
xception (Model)             (None, 1000)              22910480  
Total params: 22,910,480
Trainable params: 0
Non-trainable params: 22,910,480
_________________________________________________________________
10000/10000 [==============================] - 956s   


In [0]:
y_test.shape

(10000, 200)

#  fitting model on features saved in local for xpection model

# Experiment 1

In [0]:
def fc_layer(features=features_train,y_train = y_train):
#     if y_train is None:
#         y_train = np.load('data/saved_npy/y_train.npy')
    
    num_class = 200
    inputs = Input(features.shape[1:])
    x = inputs
    x = Dense(4096,activation = None)(x)
    x = Dense(512,activation = None)(x)
    #x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(num_class, activation='softmax')(x)
    model = Model(inputs, x)
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    h = model.fit(features_train, y_train, batch_size=64, validation_data=(x_test,y_test),epochs = 5)
    return model


In [0]:
fc_model2 = xception_from_disk()

Train on 100000 samples, validate on 10000 samples
Epoch 1/5
100000/100000 [==============================] - 165s - loss: 5.2698 - acc: 0.0083 - val_loss: 5.2987 - val_acc: 0.0050
Epoch 2/5
100000/100000 [==============================] - 177s - loss: 5.2547 - acc: 0.0091 - val_loss: 5.2993 - val_acc: 0.0050
Epoch 3/5
100000/100000 [==============================] - 184s - loss: 5.2485 - acc: 0.0093 - val_loss: 5.2991 - val_acc: 0.0050
Epoch 4/5
100000/100000 [==============================] - 187s - loss: 5.2447 - acc: 0.0096 - val_loss: 5.2988 - val_acc: 0.0050
Epoch 5/5
100000/100000 [==============================] - 184s - loss: 5.2417 - acc: 0.0095 - val_loss: 5.3014 - val_acc: 0.0050


# Experiment 2

In [0]:
def fc_layer(features=features_train,y_train = y_train):
#     if y_train is None:
#         y_train = np.load('data/saved_npy/y_train.npy')
    
    num_class = 200
    inputs = Input(features.shape[1:])
    x = inputs
    
    x = Dense(512,activation = None)(x)
    #x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(num_class, activation='softmax')(x)
    model = Model(inputs, x)
    #opt = optimizers.Adam(beta_1=.9,beta_2=.999,lr = 0.0001)
    opt = optimizers.SGD(lr=.0001)
    model.compile(optimizer=opt,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    h = model.fit(features_train, y_train, batch_size=32, validation_data=(x_test,y_test),epochs = 5)
    return model


In [0]:
fc_model2 = xception_from_disk()

Train on 100000 samples, validate on 10000 samples
Epoch 1/5
100000/100000 [==============================] - 39s - loss: 5.2990 - acc: 0.0048 - val_loss: 5.2983 - val_acc: 0.0077
Epoch 2/5
100000/100000 [==============================] - 38s - loss: 5.2990 - acc: 0.0046 - val_loss: 5.2983 - val_acc: 0.0072
Epoch 3/5
100000/100000 [==============================] - 37s - loss: 5.2989 - acc: 0.0045 - val_loss: 5.2983 - val_acc: 0.0052
Epoch 4/5
100000/100000 [==============================] - 37s - loss: 5.2988 - acc: 0.0052 - val_loss: 5.2983 - val_acc: 0.0050
Epoch 5/5
100000/100000 [==============================] - 40s - loss: 5.2989 - acc: 0.0044 - val_loss: 5.2983 - val_acc: 0.0050


# Experiment 3

In [0]:
def fc_layer(features=features_train,y_train = y_train):
#     if y_train is None:
#         y_train = np.load('data/saved_npy/y_train.npy')
    
    num_class = 200
    inputs = Input(features.shape[1:])
    x = inputs
    
    x = Dense(512,activation = None)(x)
    #x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(num_class, activation='softmax')(x)
    model = Model(inputs, x)
    #opt = optimizers.Adam(beta_1=.9,beta_2=.999,lr = 0.0001)
    opt = optimizers.SGD(lr=.0001)
    model.compile(optimizer=opt,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    h = model.fit(features_train, y_train, batch_size=64, validation_data=(x_test,y_test),epochs = 10)
    return model


In [0]:
fc_model2 = xception_from_disk()

Train on 100000 samples, validate on 10000 samples
Epoch 1/10
100000/100000 [==============================] - 20s - loss: 5.2989 - acc: 0.0044 - val_loss: 5.2983 - val_acc: 0.0056
Epoch 2/10
100000/100000 [==============================] - 20s - loss: 5.2989 - acc: 0.0043 - val_loss: 5.2983 - val_acc: 0.0052
Epoch 3/10
100000/100000 [==============================] - 20s - loss: 5.2988 - acc: 0.0043 - val_loss: 5.2983 - val_acc: 0.0070
Epoch 4/10
100000/100000 [==============================] - 20s - loss: 5.2989 - acc: 0.0047 - val_loss: 5.2983 - val_acc: 0.0067
Epoch 5/10
100000/100000 [==============================] - 20s - loss: 5.2988 - acc: 0.0041 - val_loss: 5.2983 - val_acc: 0.0068
Epoch 6/10
100000/100000 [==============================] - 21s - loss: 5.2989 - acc: 0.0045 - val_loss: 5.2983 - val_acc: 0.0068
Epoch 7/10
100000/100000 [==============================] - 21s - loss: 5.2989 - acc: 0.0045 - val_loss: 5.2983 - val_acc: 0.0066
Epoch 8/10
100000/100000 [=============

# Experiment 4

In [0]:
def fc_layer(features=features_train,y_train = y_train):
#     if y_train is None:
#         y_train = np.load('data/saved_npy/y_train.npy')
    
    num_class = 200
    inputs = Input(features.shape[1:])
    x = inputs
    
    x = Dense(256,activation = None)(x)
    #x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(num_class, activation='softmax')(x)
    model = Model(inputs, x)
    #opt = optimizers.Adam(beta_1=.9,beta_2=.999,lr = 0.0001)
    opt = optimizers.SGD(lr=.0001)
    model.compile(optimizer=opt,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    h = model.fit(features_train, y_train, batch_size=64, validation_data=(x_test,y_test),epochs = 10)
    return model


In [0]:
fc_model2 = xception_from_disk()

Train on 100000 samples, validate on 10000 samples
Epoch 1/10
100000/100000 [==============================] - 16s - loss: 5.2990 - acc: 0.0050 - val_loss: 5.2983 - val_acc: 0.0076
Epoch 2/10
100000/100000 [==============================] - 15s - loss: 5.2988 - acc: 0.0052 - val_loss: 5.2983 - val_acc: 0.0060
Epoch 3/10
100000/100000 [==============================] - 15s - loss: 5.2988 - acc: 0.0051 - val_loss: 5.2983 - val_acc: 0.0065
Epoch 4/10
100000/100000 [==============================] - 15s - loss: 5.2988 - acc: 0.0052 - val_loss: 5.2983 - val_acc: 0.0063
Epoch 5/10
100000/100000 [==============================] - 15s - loss: 5.2987 - acc: 0.0054 - val_loss: 5.2983 - val_acc: 0.0060
Epoch 6/10
100000/100000 [==============================] - 15s - loss: 5.2989 - acc: 0.0054 - val_loss: 5.2983 - val_acc: 0.0061
Epoch 7/10
100000/100000 [==============================] - 15s - loss: 5.2988 - acc: 0.0053 - val_loss: 5.2983 - val_acc: 0.0061
Epoch 8/10
100000/100000 [=============

# Experiment 5

In [0]:
def fc_layer(features=features_train,y_train = y_train):
#     if y_train is None:
#         y_train = np.load('data/saved_npy/y_train.npy')
    
    num_class = 200
    inputs = Input(features.shape[1:])
    x = inputs
    
    x = Dense(128,activation = None)(x)
    #x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(num_class, activation='softmax')(x)
    model = Model(inputs, x)
    #opt = optimizers.Adam(beta_1=.9,beta_2=.999,lr = 0.0001)
    opt = optimizers.SGD(lr=.0001)
    model.compile(optimizer=opt,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    h = model.fit(features_train, y_train, batch_size=64, validation_data=(x_test,y_test),epochs = 20)
    return model


In [0]:
fc_model2 = xception_from_disk()

Train on 100000 samples, validate on 10000 samples
Epoch 1/20
100000/100000 [==============================] - 12s - loss: 5.2987 - acc: 0.0050 - val_loss: 5.2983 - val_acc: 0.0047
Epoch 2/20
100000/100000 [==============================] - 12s - loss: 5.2988 - acc: 0.0055 - val_loss: 5.2983 - val_acc: 0.0051
Epoch 3/20
100000/100000 [==============================] - 13s - loss: 5.2988 - acc: 0.0048 - val_loss: 5.2983 - val_acc: 0.0063
Epoch 4/20
100000/100000 [==============================] - 12s - loss: 5.2987 - acc: 0.0052 - val_loss: 5.2983 - val_acc: 0.0054
Epoch 5/20
100000/100000 [==============================] - 12s - loss: 5.2988 - acc: 0.0048 - val_loss: 5.2983 - val_acc: 0.0050
Epoch 6/20
100000/100000 [==============================] - 13s - loss: 5.2987 - acc: 0.0049 - val_loss: 5.2983 - val_acc: 0.0050
Epoch 7/20
100000/100000 [==============================] - 13s - loss: 5.2988 - acc: 0.0048 - val_loss: 5.2983 - val_acc: 0.0051
Epoch 8/20
100000/100000 [=============

# Experiment 6

In [0]:
def fc_layer(features=features_train,y_train = y_train):
#     if y_train is None:
#         y_train = np.load('data/saved_npy/y_train.npy')
    
    num_class = 200
    inputs = Input(features.shape[1:])
    x = inputs
    
    #x = Dense(256,activation = None)(x)
    #x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(num_class, activation='softmax')(x)
    model = Model(inputs, x)
    opt = optimizers.Adam(beta_1=.9,beta_2=.999,lr = 0.0001)
    #opt =  optimizers.Adagrad(lr = 0.0001)
    model.compile(optimizer=opt,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    h = model.fit(features_train, y_train, batch_size=32, validation_data=(x_test,y_test),epochs = 10)
    return model



In [0]:
fc_model2 = xception_from_disk()

Train on 100000 samples, validate on 10000 samples
Epoch 1/10
100000/100000 [==============================] - 25s - loss: 5.2971 - acc: 0.0057 - val_loss: 5.2983 - val_acc: 0.0054
Epoch 2/10
100000/100000 [==============================] - 25s - loss: 5.2914 - acc: 0.0069 - val_loss: 5.2983 - val_acc: 0.0052
Epoch 3/10
100000/100000 [==============================] - 26s - loss: 5.2869 - acc: 0.0072 - val_loss: 5.2983 - val_acc: 0.0052
Epoch 4/10
100000/100000 [==============================] - 28s - loss: 5.2830 - acc: 0.0078 - val_loss: 5.2983 - val_acc: 0.0050
Epoch 5/10
100000/100000 [==============================] - 28s - loss: 5.2796 - acc: 0.0084 - val_loss: 5.2983 - val_acc: 0.0050
Epoch 6/10
100000/100000 [==============================] - 28s - loss: 5.2767 - acc: 0.0082 - val_loss: 5.2983 - val_acc: 0.0050
Epoch 7/10
100000/100000 [==============================] - 28s - loss: 5.2739 - acc: 0.0085 - val_loss: 5.2983 - val_acc: 0.0050
Epoch 8/10
100000/100000 [=============

# Conclusion: 

Experiment 3 is the best. we can add more fully connected layers and dropouts to improve the accuracy more